In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

data = pd.read_excel('datasets/nba_player_data.xlsx')

In [2]:
data.sample(10)

,Year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
3725,2017-18,Regular%20Season,204038,271,Langston Galloway,1610612765,DET,58,863,126,340,0.371,75,218,0.344,33,41,0.805,13,79,92,58,35,5,18,39,360,310,3.22,1.94
6956,2021-22,Regular%20Season,1626161,485,Willie Cauley-Stein,1610612755,PHI,20,183,16,35,0.457,1,2,0.500,1,2,0.500,13,27,40,10,6,3,7,19,34,66,1.43,0.86
984,2013-14,Regular%20Season,203515,308,Phil Pressey,1610612738,BOS,75,1132,78,253,0.308,28,106,0.264,29,45,0.644,20,85,105,241,68,5,87,97,213,354,2.77,0.78
3104,2016-17,Regular%20Season,1627751,352,Jakob Poeltl,1610612761,TOR,54,626,67,115,0.583,0,0,0.000,31,57,0.544,78,87,165,12,17,20,29,113,165,276,0.41,0.59
1418,2014-15,Regular%20Season,202695,57,Kawhi Leonard,1610612759,SAS,64,2033,394,822,0.479,67,192,0.349,202,252,0.802,85,376,461,161,148,48,97,128,1057,1300,1.66,1.53
658,2012-13,Playoffs,202933,188,Julyan Stone,1610612743,DEN,2,13,0,0,0.000,0,0,0.000,2,2,1.000,0,0,0,1,0,0,0,3,2,3,0.00,0.00
1265,2013-14,Playoffs,201175,107,Glen Davis,1610612746,LAC,13,158,25,41,0.610,0,0,0.000,0,0,0.000,14,22,36,9,3,3,7,24,50,78,1.29,0.43
6052,2020-21,Regular%20Season,1630222,361,Mason Jones,1610612755,PHI,32,334,52,123,0.423,24,66,0.364,40,64,0.625,7,48,55,42,7,1,29,24,168,149,1.45,0.24
1686,2014-15,Regular%20Season,203462,325,Archie Goodwin,1610612756,PHX,41,535,79,201,0.393,12,41,0.293,61,83,0.735,27,47,74,44,18,9,48,55,231,184,0.92,0.38
1101,2013-14,Regular%20Season,203318,425,Glen Rice,1610612764,WAS,11,109,11,37,0.297,5,17,0.294,5,7,0.714,4,16,20,7,6,1,9,7,32,29,0.78,0.67


In [3]:
data.shape

(9623, 30)

# Data cleaning & analysis preparation

In [4]:
data.drop(columns=['RANK', 'EFF'], inplace=True)

In [5]:
data['season_start_year'] = data['Year'].str[:4].astype(int)

In [ ]:
data['TEAM'].replace(to_replace=['NOP', 'NOH'], value='NO', inplace=True)

In [ ]:
data['Season_type'].replace('Regular%20Season', 'Regular', inplace=True)

In [8]:
rs_df = data[data['Season_type'] == 'Regular']
playoffs_df = data[data['Season_type'] == 'Playoffs']

In [9]:
data.columns

Index(['Year', 'Season_type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'AST_TOV', 'STL_TOV', 'season_start_year'],
      dtype='object')

In [58]:
total_cols = ['MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

## Which player stats are correlated with each other?

In [60]:
data_per_min = data.groupby(['PLAYER', 'PLAYER_ID', 'Year'])[total_cols].sum().reset_index()

for col in data_per_min.columns[4:]:
    data_per_min[col] = data_per_min[col]/data_per_min['MIN']

data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']
data_per_min['TRU%'] = 0.5 * data_per_min['PTS']/(data_per_min['FGA'] + 0.475 * data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST']/data_per_min['TOV']

data_per_min = data_per_min[data_per_min['MIN'] >= 50]
data_per_min.drop(columns='PLAYER_ID', inplace=True)

fig = px.imshow(data_per_min.select_dtypes(include=["number"]).corr())
fig.update_xaxes(tickfont=dict(size=10))
fig.update_yaxes(tickfont=dict(size=10))
fig.show()


## How are minutes played distributed?

In [61]:
fig = px.histogram(x=playoffs_df['MIN'], histnorm='percent')
fig.show()

In [62]:
def hist_data(df=rs_df, min_MIN=0, min_GP=0):
    return df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'MIN'] /\
    df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'GP']

In [63]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=hist_data(rs_df, 50, 5), 
                           histnorm='percent', 
                           name = 'Regular Season', 
                           xbins={'start':0, 'end':46, 'size':1}))
fig.add_trace(go.Histogram(x=hist_data(playoffs_df, 5, 1), 
                           histnorm='percent', 
                           name = 'Playoffs',
                           xbins={'start':0, 'end':46, 'size':1}))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)
fig.show()

In [64]:
((hist_data(rs_df, 50, 5) >= 12) & (hist_data(rs_df, 50, 5) <= 34)).mean()

0.741227367409069

In [65]:
((hist_data(playoffs_df, 5, 1) >= 12) & (hist_data(playoffs_df, 5, 1) <= 34)).mean()

0.4824626865671642

## How has the game changed over the past 10 years?

In [72]:
change_df = data.groupby('season_start_year')[total_cols].sum().reset_index()
change_df['POSS_est'] = change_df['FGA'] - change_df['OREB'] + change_df['TOV'] + 0.44 * change_df['FTA']
change_df = change_df[list(change_df.columns[0:2])+['POSS_est'] + list(change_df.columns[2:-1])]


change_df['FG%'] = change_df['FGM']/change_df['FGA']
change_df['3PT%'] = change_df['FG3M']/change_df['FG3A']
change_df['FT%'] = change_df['FTM']/change_df['FTA']
change_df['AST%'] = change_df['AST']/change_df['FGM']
change_df['FG3A%'] = change_df['FG3A']/change_df['FGA']
change_df['PTS/FGA'] = change_df['PTS']/change_df['FGA']
change_df['FG3M/FGM'] = change_df['FG3M']/change_df['FGM']
change_df['FTA/FGA'] = change_df['FTA']/change_df['FGA']
change_df['TRU%'] = 0.5 * change_df['PTS']/(change_df['FGA'] + 0.475 * change_df['FTA'])
change_df['AST_TOV'] = change_df['AST']/change_df['TOV']

change_df

,season_start_year,MIN,POSS_est,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2012,635884,248201.92,97235,215105,18808,52569,44125,58618,29237,81362,110599,57694,20376,13444,36542,52548,257403,0.452035,0.357777,0.752755,0.593346,0.244388,1.196639,0.193428,0.272509,0.529748,1.578841
1,2013,638373,254032.80,99251,218411,20480,56952,47219,62420,28669,83812,112481,57657,20156,12369,36826,54839,266201,0.454423,0.359601,0.756472,0.580921,0.260756,1.218808,0.206346,0.285791,0.536565,1.565660
2,2014,634546,253004.12,98251,219265,20724,59276,45098,60248,28566,85231,113797,57727,20261,12665,35796,53272,262324,0.448092,0.349619,0.748539,0.587546,0.270340,1.196379,0.210929,0.274773,0.529129,1.612666
3,2015,636391,258064.80,100351,222344,22524,63673,46516,61520,27426,87611,115037,58251,20562,13046,36078,53478,269742,0.451332,0.353745,0.756112,0.580473,0.286372,1.213174,0.224452,0.276688,0.536126,1.614585
4,2016,632482,258443.80,102147,223333,25408,71018,46806,60620,26470,87173,113643,59162,20143,12409,34908,52232,276508,0.457375,0.357768,0.772121,0.579185,0.317992,1.238097,0.248740,0.271433,0.548350,1.694798
5,2017,633425,260904.52,103729,225523,27530,76245,43721,57008,25397,88678,114075,60739,20181,12636,35695,52238,278709,0.459949,0.361073,0.766927,0.585555,0.338081,1.235834,0.265403,0.252781,0.551677,1.701611
6,2018,634231,268739.84,107374,233717,29817,84143,46671,60811,27128,91360,118488,64257,19940,12984,35394,55063,291236,0.459419,0.354361,0.767476,0.598441,0.360021,1.246105,0.277693,0.260191,0.554519,1.815477
7,2019,552262,234384.64,92997,202223,28032,78279,40949,52906,22802,79318,102120,55445,17368,11085,31685,47615,254975,0.459874,0.358104,0.773995,0.596202,0.387092,1.260861,0.301429,0.261622,0.560746,1.749882
8,2020,562518,235759.48,95849,205754,29549,80653,39624,50917,22918,80151,103069,57311,17491,11272,30520,45152,260871,0.465843,0.366372,0.778208,0.597930,0.391988,1.267878,0.308287,0.247465,0.567260,1.877818
9,2021,635572,264004.96,106569,231293,32733,92552,44740,57709,27052,89602,116654,64618,20006,12387,34372,52038,290611,0.460753,0.353671,0.775269,0.606349,0.400150,1.256463,0.307153,0.249506,0.561665,1.879960


In [78]:
change_per100_df = change_df.copy()
for col in change_per100_df.columns[3:18]:
    change_per100_df[col] = (change_per100_df[col] / change_per100_df['POSS_est']) * 100

change_per100_df.drop(columns=['MIN', 'POSS_est'], inplace=True)

fig = go.Figure()
for col in change_per100_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per100_df['season_start_year'], 
                             y=change_per100_df[col],
                             name = col))
    
fig.show()
change_per100_df

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2012,39.175765,86.665325,7.577701,21.179933,17.777864,23.617061,11.779522,32.780568,44.560090,23.244784,8.209445,5.416558,14.722690,21.171472,103.707095,0.452035,0.357777,0.752755,0.593346,0.244388,1.196639,0.193428,0.272509,0.529748,1.578841
1,2013,39.070152,85.977480,8.061951,22.419152,18.587757,24.571630,11.285551,32.992590,44.278140,22.696675,7.934408,4.869056,14.496553,21.587370,104.790011,0.454423,0.359601,0.756472,0.580921,0.260756,1.218808,0.206346,0.285791,0.536565,1.565660
2,2014,38.833755,86.664597,8.191171,23.428867,17.825006,23.813051,11.290725,33.687594,44.978319,22.816624,8.008170,5.005847,14.148386,21.055784,103.683687,0.448092,0.349619,0.748539,0.587546,0.270340,1.196379,0.210929,0.274773,0.529129,1.612666
3,2015,38.885970,86.158205,8.728040,24.673260,18.024930,23.838974,10.627563,33.949225,44.576788,22.572238,7.967766,5.055319,13.980210,20.722702,104.524910,0.451332,0.353745,0.756112,0.580473,0.286372,1.213174,0.224452,0.276688,0.536126,1.614585
4,2016,39.523873,86.414532,9.831151,27.479088,18.110707,23.455776,10.242072,33.729964,43.972036,22.891631,7.793958,4.801431,13.506998,20.210197,106.989605,0.457375,0.357768,0.772121,0.579185,0.317992,1.238097,0.248740,0.271433,0.548350,1.694798
5,2017,39.757456,86.438901,10.551753,29.223334,16.757471,21.850139,9.734212,33.988679,43.722891,23.280164,7.735014,4.843151,13.681250,20.021884,106.824136,0.459949,0.361073,0.766927,0.585555,0.338081,1.235834,0.265403,0.252781,0.551677,1.701611
6,2018,39.954627,86.967753,11.095117,31.310207,17.366610,22.628204,10.094521,33.995704,44.090225,23.910485,7.419815,4.831438,13.170358,20.489333,108.370981,0.459419,0.354361,0.767476,0.598441,0.360021,1.246105,0.277693,0.260191,0.554519,1.815477
7,2019,39.677088,86.278265,11.959828,33.397666,17.470855,22.572298,9.728453,33.840955,43.569408,23.655560,7.410042,4.729405,13.518377,20.314898,108.784859,0.459874,0.358104,0.773995,0.596202,0.387092,1.260861,0.301429,0.261622,0.560746,1.749882
8,2020,40.655417,87.272843,12.533536,34.209865,16.806959,21.597011,9.720924,33.996936,43.717860,24.309097,7.419002,4.781144,12.945397,19.151722,110.651330,0.465843,0.366372,0.778208,0.597930,0.391988,1.267878,0.308287,0.247465,0.567260,1.877818
9,2021,40.366287,87.609339,12.398631,35.056917,16.946651,21.859059,10.246777,33.939514,44.186291,24.476055,7.577888,4.691957,13.019452,19.710993,110.077856,0.460753,0.353671,0.775269,0.606349,0.400150,1.256463,0.307153,0.249506,0.561665,1.879960


In [79]:
change_per48_df = change_df.copy()
for col in change_per48_df.columns[2:18]:
    change_per48_df[col] = (change_per48_df[col] / change_per48_df['MIN']) * 48 * 5

change_per48_df.drop(columns=['MIN'], inplace=True)

fig = go.Figure()
for col in change_per48_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per48_df['season_start_year'], 
                             y=change_per48_df[col],
                             name = col))
    
fig.show()

## Compare Regular Season to Playoff

In [ ]:
rs_change_df = rs_df.groupby('season_start_year')[total_cols].sum().reset_index()
playoffs_change_df = playoffs_df.groupby('season_start_year')[total_cols].sum().reset_index()

for i in [rs_change_df, playoffs_change_df]:
    i['POSS_est'] = i['FGA'] - i['OREB'] + i['TOV'] + 0.44 * i['FTA']
    i['POSS_per48'] = (i['POSS_est']/i['MIN']) / 48

    i['FG%'] = i['FGM']/i['FGA']
    i['3PT%'] = i['FG3M']/i['FG3A']
    i['FT%'] = i['FTM']/i['FTA']
    i['AST%'] = i['AST']/i['FGM']
    i['FG3A%'] = i['FG3A']/i['FGA']
    i['PTS/FGA'] = i['PTS']/i['FGA']
    i['FG3M/FGM'] = i['FG3M']/i['FGM']
    i['FTA/FGA'] = i['FTA']/i['FGA']
    i['TRU%'] = 0.5 * i['PTS']/(i['FGA'] + 0.475 * i['FTA'])
    i['AST_TOV'] = i['AST']/i['TOV']

    for col in total_cols:
        i[col] = 100 * (i[col]/i['POSS_est'])
    